# LCT-AGI

In [1]:
%%capture
!pip install langchain
!pip install openai

In [11]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, tool

### 参照するLLM

In [8]:
#input = 'trainig XGBoost'
input = 'training LightGBM'
exec_list = [
    'training LightGBM'
]

In [9]:
def deciding_agent(input_, exec_list_):
    print("deciding agent")
    
    decide_llm = OpenAI(temperature=0,model_name="gpt-4")

    i = ''
    for i in exec_list:
        i = i + ','
    i = '[' + i + ']'

    prompt = """
    You are an agent that determines if the input task is executable. 
    All you can do is to be included in {exec_list}. 
    Answer "Yes" if you can perform the task, or "No" if you cannot.
    ------
    The entered task is:{input}
    ------
    """.format(exec_list = i,input = input_)

    return decide_llm(prompt)

In [10]:
decide_agent(input, exec_list)

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:165: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:672: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'Yes'

### 道具をつくるLLM

In [23]:
def toolmaking_agent(input_):
    print("toolmaking agent")

    toolmaking_llm = OpenAI(temperature=0, model_name="gpt-4")


    prompt = """
    Create a python function that can execute {input} with a single string as input.
    The code below is the code created with the input "multiply two numbers".
    The output is only the code of the created python.
    -----------------------------------
    @tool('new_tool')
    def new_tool(query: str) -> str:
        'Used for multiplication. The input is two numbers. For example, if you want to multiply 1 by 2, the input is '1,2'.'
        a, b = query.split(",")
        c = int(a) * int(b)
        return c
    """.format(input = input_)

    code = toolmaking_llm(prompt)

    #name = query.replace(' ','_').lower()

    #bos = "\n@tool('{name}_tool)\n"\
    #        "def {name}_tool(query: str) -> str:\n"\
    #        "    'useful for {query}'"
    
    #code = textwrap.indent(textwrap.dedent(code)[:], '    ')

    #eos = "\n    result = 'finish {name}\n"\
    #        "    return result" 

    #result = bos.format(name=name, query=query) + code + eos.format(name=name)

    file = open('/home/langchain-tools/examples/new_tool.py', mode='w')
    file.write(code)
    file.close()

    return 'finish'

In [24]:
input = 'Output multiples of 3 up to 20'
toolmaking_agent(input)

toolmaking agent


'finish'

### 実行するLLM

In [ ]:
def executing_agent(input_, tools_):
    print("executing agent")

    excute_llm = OpenAI(temperature=0, model_name="gpt-4")

    agent = initialize_agent(tools_, excute_llm, agent="zero-shot-react-description", verbose=True)

    agent.run(input_)

    return

### 全体

In [ ]:
def main(input_, exec_list_, tools_):
    
    output = deciding_agent(input_, exec_list_)

    if output == "Yes":
        print('Execute it as it is executable without creating a tool.')
        executing_agent(input_, tools_)

    elif output == "No":
        print('It is necessary to create a tool, so run it after creating the tool.')
        toolmaking_agent(input_)

        tools.append(Tool(name = f'{new_tool}', func = new_tool.new_tool, description=''))

        executing_agent(input_, tools_)
    
    return

In [4]:
llm = OpenAI(temperature=0,model_name="gpt-4")
llm("面白いことを言って")

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:165: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:672: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'ある日、カエルとヘビが一緒に映画を見に行きました。映画が始まると、カエルはポップコーンを食べ始めました。ヘビはカエルを見て言いました、「君は本当にポップコーンが好きだね。でも、どうして飛び跳ねないの？」カエルは笑って答えました、「それは、ポップコーンが私のお腹の中に入ると、もうポップしなくなるからさ！」'